In [3]:
import glob
import os
from paths import *

In [4]:
def find_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, prefix + '*')
    matching_files = glob.glob(pattern)
    
    # Sort the matching files alphabetically
    matching_files.sort()
    
    return matching_files

In [5]:
directory_path = graph_dir
prefix = 'Left_B_02_034-TRY-0LGN-101'

result = find_files_with_prefix(directory_path, prefix)
print(result)

['../src/graph\\Left_B_02_034-TRY-0LGN-101_000000.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000001.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000002.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000003.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000004.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000005.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000006.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000007.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000008.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000009.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000010.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000011.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000012.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000013.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000014.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000015.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000016.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_00001